<a href="https://colab.research.google.com/github/Tyanakai/religious_art/blob/main/religious_art_infere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>宗教画テーマの分類　推論</h1>


# 1.はじめに
本ノートブックでは、
1. [religious_art_train.ipynb](https://github.com/Tyanakai/religious_art/blob/main/religious_art_train.ipynb)において訓練し保存したモデルを使用しout of fold trainデータ、及びtestデータに対し予測を行います。<br>
2. 予測値(probability)にアンサンブル手法を適用し、評価します。
3. アンサンブル手法から計算した予測値から、擬似ラベルを作成します。<br>

尚、colabratory上で、ランタイムのタイプをGPUに設定した状態での実行を想定しています。

# 2.事前に完了していること
- [religious_art_train.ipynb](https://github.com/Tyanakai/religious_art/blob/main/religious_art_train.ipynb)を実行し、EfficientNetB4, EfficientNetB7, EfficientNetV2-Sモデルを訓練、保存(.h5)していること。

# 3.環境準備
実行のための環境を構築します。

## 3.1 GPU

In [1]:
!nvidia-smi

Tue Nov 16 05:27:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 3.2 ライブラリ

In [2]:
!pip install -q efficientnet
!pip install -q tensorflow-addons

import datetime
import json
import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
import tensorflow_hub as hub
from tqdm.notebook import tqdm

     |████████████████████████████████| 50 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 5.3 MB/s 


## 3.3 ハイパーパラメータ
|パラメータ名|説明|
|:-|:-|
|time_jp_list|使用するモデル訓練した時刻。記録を呼び出す際に用いる|
|-|-|
|train_imgs_file|訓練画像データのファイル名|
|train_labels_file|訓練ラベルデータのファイル名|
|test_imgs_file|テスト画像データのファイル名|
|original_size|画像の原寸。正方形の一辺の長さ|
|tta|test time augmentを行うか否か|
|tta_iters|test time augmentで、一つのテストデータに対し作る拡張データの数|
|num_ops|test time augmentを行う際、一度の拡張で適用する手法の数| 
|magnitude|test time augmentを行う際、適用する拡張手法の度合|
|-|-|
|solo_submit|アンサンブルしない予測値で提出ファイルを作るか否か|
|pseudo_threshold|閾値。予測値 > 閾値となるデータだけ擬似ラベルを作成する|

In [3]:
class Config:
    # モデルを訓練した時刻のリスト
    time_jp_list = [11121537,11141521,11141537] #@param

    batch_size = 8 #@param {type:"raw"} [1,2,4,8,16]
    n_folds = 5
    seed = 21 #@param
    exclusive = True #@param {"type":"boolean"}

    train_imgs_file = "christ-train-imgs.npz"
    train_labels_file = "christ-train-labels.npz"
    test_imgs_file = "christ-test-imgs.npz"
    original_size = 224
    n_classes = 13

    tta = True #@param {"type":"boolean"}
    tta_iters = 2 #@param
    num_ops = 5 #@param {type:"slider", min:0, max:20, step:1}
    magnitude = 5 #@param {type:"slider", min:0, max:10, step:0.5}    

    solo_submit = True #@param {"type":"boolean"}
    pseudo_threshold = 0.13 #@param
    debug = True #@param {"type":"boolean"}

if Config.debug:
    Config.n_folds = 2

AUTOTUNE = tf.data.experimental.AUTOTUNE


## 3.4 path

In [4]:
DRIVE = "/content/drive/MyDrive/portforio/religious_art"
GS = "gs://colab_test2/" # Google Cloud Storage 

INPUT = os.path.join(DRIVE, "input")
OUTPUT = os.path.join(DRIVE, "output")
RECORD = os.path.join(DRIVE, "model", "record")
SUBMIT = os.path.join(DRIVE, "submit")
PROB = os.path.join(DRIVE, "prob") # 予測確率値(probability)を保存するフォルダ

GS_TRAIN = os.path.join(GS, "train_image")

# efficientnet-v2のdownload元
hub_url = ("https://tfhub.dev/google/imagenet/"
           "efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2")

# 4.データ準備

## 4.1 取得

In [5]:
def get_data(file_name):
    """
    データを取得する。
    """
    data = np.load(os.path.join(INPUT, file_name))["arr_0"]

    file_idx = np.arange(data.shape[0])
    if Config.exclusive:
        ex_file_idx = make_exclusive(file_idx)
        data = data[ex_file_idx]
    
    return data

## 4.2 加工

In [6]:
 def skf(x_train, y_train, n_folds=Config.n_folds, random_state=Config.seed):
     """
     訓練データを層化K分割し、リスト化したindexを取得する。
     """
     kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
     return list(kf.split(x_train, y_train))

In [7]:
def make_exclusive(index_list):
    """
    index_listからdup_idxにあるindexを取り除く
    """
    dup_idx = np.load(os.path.join(OUTPUT, "dup_idx.npy"))
    exclusive_idx_set = set(idx for idx in index_list) - set(dup_idx)
    return np.array(list(exclusive_idx_set))

In [8]:
def smooth_label(label, factor=0.1):
    """
    label smoothingを適用する。
    """
    labels = np.array((1 - factor) * label + (factor / label.shape[1]))
    return labels.astype(np.float32)

## 4.3 tf.data.Dataset

In [9]:
class ImageProcessor():
    """
    np.arrayを受け取り、画像を処理してtf.data.Dataset形式で出力

    Parameters
    --------------
        num_ops : int
            random augmentを行う際、適用する拡張手法の数
        magnitude : int
            random augmentを行う際、適用する拡張手法の度合
        image_size : int
            datasetの出力画像サイズ(＝訓練モデルの入力画像サイズ)
    """
    def __init__(self, 
                 num_ops=Config.num_ops, 
                 magnitude=Config.magnitude, 
                 image_size=380):
        self.num_ops = num_ops
        self.magnitude = magnitude
        self.image_size = image_size

    def load_image(self, image, label=None):
        """
        np.arrayを受け取り、float32型のtensorを返す。
        """
        image = tf.cast(image, tf.float32)
        return image, label

    def mix_up(self, ds1, ds2, alpha=0.2):
        """
        拡張法：2枚の画像とそのラベルをランダムな割合で合成する。
        """
        lam = tf.random.uniform(shape=[], maxval=0.11, dtype=tf.float32)
        image = tf.add(tf.multiply(ds1[0], (1 - lam)), tf.multiply(ds2[0], lam))
        label = tf.add(tf.multiply(ds1[1], (1 - lam)), tf.multiply(ds2[1], lam))
        return image, label

    def random_rotate(self, image, label=None):
        """
        拡張法：ランダムな角度(度数法)で画像を回転する。
        """
        level = np.pi * self.magnitude / 180 # 度数法を孤度法へ変換

        angle = tf.random.uniform(
            shape=[],minval=-level, maxval=level, dtype=tf.float32)
        image = tfa.image.rotate(
            image, angle, 
            fill_mode="nearest"
            )
        return image, label

    def random_cutout(self, image, label=None):
        """
        拡張法：画像のランダムな一部をに黒塗りにする。
        黒塗りは正方形となり、一辺の長さが偶数でなくてはならない。
        """
        mask = int(7 + 3 * self.magnitude) * 2
        image = tf.reshape(image, [1, Config.original_size, Config.original_size, 3])
        image = tfa.image.random_cutout(image, (mask, mask))
        return image[0], label

    def random_brightness(self, image, label=None):
        """
        拡張法：輝度をランダムに変化させる。
        """
        bright_delta = 0.05 * self.magnitude
        image = tf.image.random_brightness(image, bright_delta)
        return image, label

    def random_contrast(self, image, label=None):
        """
        拡張法：コントラストをランダムに変化させる。
        """
        lower = 0.8 - 0.06 * self.magnitude
        upper = 1.1 + 0.09 * self.magnitude
        image = tf.image.random_contrast(image, lower=lower, upper=upper)
        return image, label
    
    def random_saturation(self, image, label=None):
        """
        拡張法：彩度をランダムに変化させる。
        """
        lower = 0.8 - 0.079 * self.magnitude
        upper = 1.1 + 0.09 * self.magnitude
        image = tf.image.random_saturation(image, lower=lower, upper=upper)
        return image, label
    
    def random_hue(self, image, label=None):
        """
        拡張法：色相をランダムに変化させる。
        """
        hue_delta = 0.01 + 0.005 * self.magnitude #0819 0.004->0.005
        image = tf.image.random_hue(image, hue_delta)
        return image, label
       
    def random_flip(self, image, label=None):
        """
        拡張法：ランダムな割合で左右を反転する。
        """
        image = tf.image.random_flip_left_right(image)
        return image, label

    def random_zoom(self, image, label=None):
        """
        拡張法：ランダムな焦点にズームする。
        """
        zoom_rate = 0.05 + 0.035 * self.magnitude
        crop_size = tf.random.uniform(
            shape=[], 
            minval=int(Config.original_size*(1. - zoom_rate)), 
            maxval=Config.original_size, 
            dtype=tf.int32)
        image = tf.image.random_crop(image, size=[crop_size, crop_size, 3])
        image = tf.image.resize(image, size=[Config.original_size, Config.original_size])
        return image, label

    def random_sharpness(self, image, label=None):
        """
        拡張法：輪郭の強調感をランダムに変化させる。
        """
        factor = 0.01 + 0.089 * self.magnitude # 0819 0.009->0.089
        factor = tf.random.uniform([], minval=0, maxval=factor)
        image = tfa.image.sharpness(image, factor)
        return image, label

    def random_shear_x(self, image, label=None):
        """
        拡張法：ランダムに水平せん断写像を行う。
        """
        level = 0.02 + 0.015 * self.magnitude # 0819 0.018->0.015
        level = tf.random.uniform([], minval=-level, maxval=level)
        image = tfa.image.shear_x(image, level, [128,128,128])
        return image, label

    def random_shear_y(self, image, label=None):
        """
        拡張法：ランダムに鉛直せん断写像を行う。
        """
        level = 0.02 + 0.015 * self.magnitude # 0819 0.018->0.015
        level = tf.random.uniform([], minval=-level, maxval=level)
        image = tfa.image.shear_y(image, level, [128,128,128])
        return image, label

    def random_solarize_add(self, image, label=None):
        """
        拡張法：ランダムな露出効果を付加する。
        """
        addition = np.round(1 + 0.9 * self.magnitude)
        threshold = 1 + 3.1 * self.magnitude
        addition = tf.random.uniform([], minval=-addition, maxval=addition+1, dtype=tf.int64)
        threshold = tf.random.uniform([], minval=0, maxval=threshold)
        # tf.print("addition:", addition)
        # tf.print("threshold:", threshold)
        added_image = tf.add(tf.cast(image, tf.int64), addition)
        added_image = tf.cast(tf.clip_by_value(added_image, 0, 255), tf.float32)
        image = tf.where(image < threshold, added_image, image)
        return image, label

    def random_posterize(self, image, label=None):
        """
        拡張法：ランダムに色数を削減する。
        """
        shift = np.round(0.6 * self.magnitude)
        shift = tf.random.uniform([], minval=0, maxval=shift+1, dtype=tf.int32)
        # tf.print("shift:", shift)
        image = tf.cast(image, tf.int32)
        image =  tf.bitwise.left_shift(tf.bitwise.right_shift(image, shift), shift)
        image = tf.cast(image, tf.float32)
        return image, label

    def identity(self, image, label=None):
        """
        恒等関数
        """
        return image, label

    def resize_divide_image(self, image, label=None):
        """
        モデルの入力形式に画像を調整する。
        """
        image = tf.image.resize(image, size=[self.image_size, self.image_size])
        image = tf.math.divide(image, 255)
        return image, label

    def test_dataset(self, file_paths, labels=None, batch_size=Config.batch_size):
        val_ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
        val_ds = val_ds.batch(batch_size)
        val_ds = val_ds.map(self.load_image, num_parallel_calls=AUTOTUNE)
        val_ds = val_ds.map(self.resize_divide_image, num_parallel_calls=AUTOTUNE)
        val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
        return val_ds

    def tta_dataset(self, file_path, label=None, batch_size=Config.batch_size):
        """
        test time augment手法を用いる際のdataset
        """
        tta_ds = tf.data.Dataset.from_tensor_slices((file_path, label))
        tta_ds = tta_ds.map(self.load_image, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.map(self.random_flip, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.map(self.random_zoom, num_parallel_calls=AUTOTUNE) 
        tta_ds = tta_ds.map(self.random_brightness, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.map(self.random_contrast, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.map(self.random_saturation, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.map(self.random_hue, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.map(self.random_cutout, num_parallel_calls=AUTOTUNE)
        # tta_ds = tta_ds.map(self.random_sharpness, num_parallel_calls=AUTOTUNE)   
        # tta_ds = tta_ds.map(self.random_solarize_add, num_parallel_calls=AUTOTUNE)
        # tta_ds = tta_ds.map(self.random_rotate, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.batch(batch_size)
        tta_ds = tta_ds.map(self.resize_divide_image, num_parallel_calls=AUTOTUNE)
        tta_ds = tta_ds.prefetch(buffer_size=AUTOTUNE)
        return tta_ds

# 5.モデル準備
モデル全体の構造を定義し取得します。

In [10]:
def get_pretrained_model(model_name):
    """
    使用する事前学習されたmodelを取得する。
    """       
    if model_name=="efn_b4":
        pretrained_model = efn.EfficientNetB4(
            weights='noisy-student', 
            include_top=False)
        
    elif model_name=="efn_b7":
        pretrained_model = efn.EfficientNetB7(
            weights='noisy-student', 
            include_top=False)
    
    elif "v2" in model_name:
        pretrained_model = hub.KerasLayer(hub_url)

    pretrained_model.trainable = True

    return pretrained_model


def build_model(model_name, image_size):
    """
    modelの全体構造を定義する。
    """
    pretrained_model = get_pretrained_model(model_name)
    
    input = tf.keras.layers.Input(shape=(image_size, image_size, 3))
    x = pretrained_model(input)
    if "v2" not in model_name:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    elif "v2" in model_name:
        x = tf.keras.layers.Dropout(rate=0.2)(x)
    pred = tf.keras.layers.Dense(Config.n_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input, outputs=pred)
    
    return model

# 6.推論

## 6.1 推論関数
推論の手順を定義します。<br>

In [11]:
def predict_oof(time_jp):
    """
    訓練した時刻を引数として渡し、記録を呼び出す。
    それに基づいてout of fold のデータに対する予測(probability)を行う。
    """
    print("oof predict\n")
    # 記録の呼び出し
    with open(os.path.join(RECORD, f"record{time_jp}.json"), mode="r") as f:
        record = json.load(f)

    # データ準備
    imgs = get_data(Config.train_imgs_file) # shape:(654, 224, 224, 3)
    labels = get_data(Config.train_labels_file) # shape:(654,)

    image_processor = ImageProcessor(image_size=record["image_size"])

    # model準備
    model = build_model(record["model_name"], record["image_size"])

    # oof予測値(probability)を格納
    oof_prob = np.zeros([imgs.shape[0], Config.n_classes])


    # fold毎にmodel_pathとoofのindexを取得(訓練時のseedを用いてk分割を再現)
    for model_path, (_, val_idx) in zip(tqdm(record["model_path"]), 
                                        skf(imgs, labels, random_state=record["seed"])):
        K.clear_session()
        model.load_weights(model_path)
        
        # dataset準備
        oof_ds = image_processor.test_dataset(imgs[val_idx])

        # 予測
        pred = model.predict(oof_ds)
        oof_prob[val_idx] = pred

    prob_df = pd.DataFrame(oof_prob)
    prob_df.to_csv(os.path.join(PROB, f"model{time_jp}_oof_prob.csv"),
                   index=False)

In [12]:
def predict_test(time_jp):
    """
    訓練した時刻を引数として渡し、記録を呼び出す。
    それに基づいてtestデータに対する予測を行い、提出ファイルを作成保存する。
    """
    print("test predict\n")
    with open(os.path.join(RECORD, f"record{time_jp}.json"), mode="r") as f:
        record = json.load(f)

    # dataset準備
    imgs = get_data(Config.test_imgs_file)
    image_processor = ImageProcessor(image_size=record["image_size"])

    test_ds = image_processor.test_dataset(imgs)
    
    # model準備
    model = build_model(record["model_name"], record["image_size"])

    # 予測値(probability)を格納
    test_prob = []

    # fold毎に保存したmodelで予測
    for model_path in tqdm(record["model_path"]):
        K.clear_session()
        model.load_weights(model_path)

        # 予測
        test_prob.append(model.predict(test_ds, verbose=1))

    # 予測値(probability)の保存
    mean_prob = np.mean(test_prob, axis=0)
    prob_df = pd.DataFrame(mean_prob)
    prob_df.to_csv(os.path.join(PROB, f"model{time_jp}_test_prob.csv"),
                   index=False)

    # 提出ファイルの作成保存
    if Config.solo_submit:
        predict = np.argmax(mean_prob, axis=-1)

        submit_df = pd.DataFrame({"id":np.arange(1, imgs.shape[0]+1),
                                  "y":predict})
        submit_df.to_csv(os.path.join(SUBMIT, 
                                      f"model{time_jp}_submit.csv"),
                         index=False)

In [13]:
def predict_tta(time_jp):
    """
    訓練した時刻を引数として渡し、記録を呼び出す。
    それに基づき、test time augmentation手法を用いて、testデータに対する予測、
    提出ファイルの作成保存を行う。
    """
    print("tta predict\n")
    with open(os.path.join(RECORD, f"record{time_jp}.json"), mode="r") as f:
        record = json.load(f)

    # dataset準備
    imgs = get_data(Config.test_imgs_file)
    image_processor = ImageProcessor(magnitude=Config.magnitude,
                                     image_size=record["image_size"])

    tta_ds = image_processor.tta_dataset(imgs)

    # model準備
    model = build_model(record["model_name"], record["image_size"])

    # 予測値(probability)を格納
    test_prob = []

    # fold毎に保存したmodelで予測
    for model_path in tqdm(record["model_path"]):
        K.clear_session()
        model.load_weights(model_path)

        for _ in range(Config.tta_iters):         
            prob = model.predict(tta_ds, verbose=1)
            test_prob.append(prob)

    # 予測値(probability)の保存
    mean_prob = np.mean(test_prob, axis=0)
    prob_df = pd.DataFrame(mean_prob)
    prob_df.to_csv(os.path.join(PROB, f"model{time_jp}_test_tta_prob.csv"),
                   index=False)

    # 提出ファイルの作成保存
    if Config.solo_submit:
        predict = np.argmax(mean_prob, axis=-1)

        submit_df = pd.DataFrame({"id":np.arange(1, imgs.shape[0]+1),
                                  "y":predict})
        submit_df.to_csv(os.path.join(SUBMIT, 
                                      f"model{time_jp}_tta_submit.csv"),
                         index=False)


## 6.2 実行

In [14]:
def infere():
    for time_jp in tqdm(Config.time_jp_list):
        predict_oof(time_jp)
        predict_test(time_jp)
        if Config.tta:
            predict_tta(time_jp)

In [15]:
infere()

  0%|          | 0/3 [00:00<?, ?it/s]

oof predict

71688192/71678424 [==============================] - 1s 0us/step


  0%|          | 0/2 [00:00<?, ?it/s]

test predict



  0%|          | 0/2 [00:00<?, ?it/s]

...

test predict



  0%|          | 0/2 [00:00<?, ?it/s]

62/62 [==============================] - 89s 1s/step
tta predict



  0%|          | 0/2 [00:00<?, ?it/s]

62/62 [==============================] - 87s 1s/step


# 7.アンサンブル
保存した予測値に対してアンサンブル手法を適用します。<br>
又、アンサンブルの結果得た予測値を元に擬似ラベルを作成します。

## 7.1 評価指標
訓練でも用いたfocal loss関数をアンサンブル手法を評価する為に使用します。

In [16]:
def categorical_focal_loss(alpha, gamma=2.):
    
    alpha = np.array(alpha, dtype=np.float32)

    def _categorical_focal_loss(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        return K.mean(K.sum(loss, axis=-1))

    return _categorical_focal_loss

## 7.2 予測値準備
推論実行で保存した予測値を読み込みます。

In [17]:
def get_prob():
    """
    保存した予測値を取得する。
    """
    oof_prob_list = []
    test_prob_list = []
    tta_prob_list = []

    for time_jp in Config.time_jp_list:
        oof_prob_path = os.path.join(PROB, f"model{time_jp}_oof_prob.csv")
        oof_prob = pd.read_csv(oof_prob_path)
        oof_prob_list.append(oof_prob.values)

        test_prob_path = os.path.join(PROB, f"model{time_jp}_test_prob.csv")
        test_prob = pd.read_csv(test_prob_path)
        test_prob_list.append(test_prob.values)    

        if Config.tta:
            tta_prob_path = os.path.join(PROB, f"model{time_jp}_test_tta_prob.csv")
            tta_prob = pd.read_csv(tta_prob_path)
            tta_prob_list.append(tta_prob.values)

    return np.array(oof_prob_list), np.array(test_prob_list), np.array(tta_prob_list)

## 7.3 アンサンブル関数(加重平均)
アンサンブル手法として、各モデルの予測値の加重平均を計算します。

In [18]:
def get_3d_weight():
    """
    ３つの要素からなる0.1刻みの重みリストを取得する。
    重みの合計値は1.0となる。ex) [0.4, 0.3, 0.3]
    """
    weight_list = []
    for i in range(11):
        for j in range(11-i):
            for k in range(11-i-j):
                if i+j+k==10:
                    weight_list.append([i,j,k])
    weight_list = np.array(weight_list) * 0.1
    return weight_list


def eval_weighted_mean(prob_list):
    """
    予測値の加重平均を算出し、評価する。
    """

    weight_list = get_3d_weight()
    acc_list = []
    loss_list = []

    # 正解ラベルの準備
    labels = get_data(Config.train_labels_file)
    oh_labels = pd.get_dummies(labels).values.astype(np.float32)
    smooth_labels = smooth_label(oh_labels)

    # 評価の為の損失関数
    loss_fn = categorical_focal_loss(alpha=1)

    for weight in weight_list:

        # 重みと予測値の積
        weighted_prob = sum([w * prob for w, prob in zip(weight, prob_list)])

        prediction = np.argmax(weighted_prob, axis=1)
        acc_list.append(metrics.accuracy_score(labels, prediction))

        loss = loss_fn(smooth_labels, weighted_prob).numpy()
        loss_list.append(loss)

    eval_df = pd.DataFrame()
    eval_df["acc"] = acc_list
    eval_df["loss"] = loss_list
    eval_df["weight"] = list(map(lambda x: str(x), weight_list))
     
    return eval_df

## 7.4 擬似ラベル作成
アンサンブルの予測値から擬似ラベルを作成します。

In [19]:
def make_pseudo_data(prob_list):
    """
    擬似(ソフト)ラベルを保存する。
    """
    # probabilityの高いindexを抽出
    pseudo_idx = np.unique(np.where(prob_list > Config.pseudo_threshold)[0])

    # 抽出したprobabilityをcsv形式で保存
    pseudo_prob = prob_list[pseudo_idx]
    pseudo_soft_df = pd.DataFrame(pseudo_prob)
    pseudo_soft_df.index = pseudo_idx
    pseudo_soft_df.to_csv(os.path.join(OUTPUT, "pseudo_soft.csv"))

## 7.5 実行

In [20]:
def ensemble():
    # 予測値の取得
    oof_prob_list, test_prob_list, tta_prob_list = get_prob()

    # oofデータで加重平均を評価
    eval_df = eval_weighted_mean(oof_prob_list)

    # oofデータでの最適な重みを取得
    eval_df["acc_rank"] = np.argsort(np.argsort(-eval_df.acc.values)) # accuracyの順位
    eval_df["loss_rank"] = np.argsort(np.argsort(eval_df.loss.values)) # lossの順位
    eval_df["total_rank"] = eval_df.acc_rank + eval_df.loss_rank
    best_index = eval_df["total_rank"].idxmin()

    best_weight = get_3d_weight()[best_index]
    print(best_weight)


    # 最適な重みでtestデータの予測値を加重平均
    weighted_prob = sum(w*p for w, p in zip(best_weight, test_prob_list))

    
    # 提出ファイルの作成保存
    time_str = "-".join(str(t) for t in Config.time_jp_list)
    predict = np.argmax(weighted_prob, axis=1)

    submit_df = pd.DataFrame(
        {"id": np.arange(1, test_prob_list.shape[1]+1), "y": predict}
        )
    submit_df.to_csv(
        os.path.join(SUBMIT, f"ensemble{time_str}_submit.csv"), 
        index=False
        )
    
    # 擬似ラベルの作成保存
    make_pseudo_data(weighted_prob)
    
    if Config.tta:
        weighted_prob_tta = sum(w*p for w, p in zip(best_weight, tta_prob_list))
        predict = np.argmax(weighted_prob_tta, axis=1)
        submit_df["y"] = predict
        submit_df.to_csv(
            os.path.join(SUBMIT, f"ensemble{time_str}_tta_submit.csv"), 
            index=False
            )
        make_pseudo_data(weighted_prob_tta)

In [21]:
ensemble()

[0.8 0.2 0. ]
